For our objective, it's particularly important to focus not just on overall accuracy but also on the model's ability to correctly identify the high-risk patients (Class 1). This is why metrics like recall, precision, ROC AUC, and F1-score for the mortality class are essential.

Model Interpretation: For some of the more elementary models we performed feature importance analysis,to provide us insights into which factors are most predictive of near-term mortality, helping clinicians understand key risk factors.

Use in Clinical Settings: Such models, if deployed in clinical settings, can assist healthcare professionals in making informed decisions, but they should be used as a supplement to, not a replacement for, clinical judgment and expertise.


Load the features and labels for the specific task (in-hospital mortality at 48 hours). This will involve importing necessary packages and reading the data files.

In [4]:
import sparse
import json
import pandas as pd

# Replace {task} with the specific task, e.g., 'mortality_48h'
s = sparse.load_npz('Features/s.npz').todense()
X = sparse.load_npz('Features/X.npz').todense()

s_feature_names = json.load(open('Features/s.feature_names.json', 'r'))
X_feature_names = json.load(open('Features/X.feature_names.json', 'r'))

df_pop = pd.read_csv('Population/mortality_48h.csv')


Why I decided to combine the time-invariant and time-dependent features for this file

Holistic View of Data: Combining both time-invariant and time-dependent features provides a more comprehensive view of each patient's record. Time-invariant data often includes critical baseline information, while time-dependent data captures dynamic changes. Using both together allows the model to make more informed predictions.

Model Limitations: Most traditional machine learning models, including Random Forest, expect input data in a tabular format (2D). They are not designed to natively handle 3D tensors (such as those including time steps as a separate dimension). Therefore, reshaping and combining the data into a 2D format makes it compatible with these models.

Feature Relationships: By combining the features, you allow the model to learn from both static patient characteristics and their changes over time, potentially uncovering important relationships between these features.

Simplifies Model Training: Having a single dataset simplifies the model training process. It allows you to feed all relevant information into the model at once, rather than trying to integrate separate models for time-invariant and time-dependent data.

Practicality in Model Evaluation and Deployment: With a single model that considers all features, it's easier to evaluate its performance, tune it, and potentially deploy it in a real-world setting.

Caveats
Dimensionality: The trade-off is that combining features can lead to high dimensionality, as you've experienced. This requires careful handling to ensure the model isn't overwhelmed by too many features, some of which might be noisy or irrelevant.
Feature Importance Interpretation: When time-dependent features are expanded across time steps, interpreting the model's feature importance can become more complex.
Alternative Approaches
In some cases, alternative modeling approaches that can natively handle time-series data, like Recurrent Neural Networks (RNNs) or Long Short-Term Memory (LSTM) networks, might be used. However, these models are more complex and computationally intensive compared to Random Forest.

In [17]:
import numpy as np

# Reshape X from 3D tensor to 2D matrix
# The new shape should be (number of patients, time * features)
# Assuming the first dimension of X is the number of patients
num_patients = X.shape[0]
time_steps = X.shape[1]
num_features_X = X.shape[2]

X_reshaped = X.reshape(num_patients, time_steps * num_features_X)

# Concatenate s (time-invariant) and X_reshaped (time-dependent) features
# Ensure that both s and X_reshaped have the same number of rows (patients)
combined_features = np.concatenate([s, X_reshaped], axis=1)

# Now combined_features is your full feature set for the model
print(f"Shape of time-invariant features (s): {s.shape}")
print(f"Shape of time-dependent features (X) after reshaping: {X_reshaped.shape}")
print(f"Shape of combined features: {combined_features.shape}")

# Expanding time-dependent feature names for each time step
expanded_X_feature_names = [f"{name}_time{t}" for t in range(time_steps) for name in X_feature_names]

# Combine with time-invariant feature names
total_feature_names = s_feature_names + expanded_X_feature_names

# Check if the numbers match now
print(f"Total number of features in combined dataset: {combined_features.shape[1]}")
print(f"Total number of feature names: {len(total_feature_names)}")


Shape of time-invariant features (s): (8577, 96)
Shape of time-dependent features (X) after reshaping: (8577, 350736)
Shape of combined features: (8577, 350832)
Total number of features in combined dataset: 350832
Total number of feature names: 350832


Based on the contents of the mortality_48h.csv file provided. The partition column categorizes each record into different sets: typically 'train', 'test', and sometimes 'validation'.

In [18]:
# 'combined_features' is combined dataset from time-invariant and time-dependent data

# Separating data based on the predefined splits
train_data = combined_features[df_pop['partition'] == 'train']
test_data = combined_features[df_pop['partition'] == 'test']
# If there's a validation set
# validation_data = combined_features[df_population['partition'] == 'validation']

# Separating labels
train_labels = df_pop[df_pop['partition'] == 'train']['mortality_LABEL']
test_labels = df_pop[df_pop['partition'] == 'test']['mortality_LABEL']
validation_labels = df_pop[df_pop['partition'] == 'val']['mortality_LABEL']


In [19]:
# After loading and combining features and labels

# Check 1: Ensure matching indices between df_pop and combined_features
if len(df_pop) != combined_features.shape[0]:
    raise ValueError("Mismatch in number of rows between df_pop and combined features")

# Check 2: Check the dimensions of split data
if train_data.shape[0] != train_labels.shape[0]:
    raise ValueError("Mismatch in number of rows between train_data and train_labels")

if test_data.shape[0] != test_labels.shape[0]:
    raise ValueError("Mismatch in number of rows between test_data and test_labels")

# If you have a validation set
if validation_labels.shape[0] != df_pop[df_pop['partition'] == 'val'].shape[0]:
    raise ValueError("Mismatch in number of rows between validation_data and validation_labels")

# Check 3: Verify feature concatenation
expected_feature_count = s.shape[1] + X_reshaped.shape[1]
if combined_features.shape[1] != expected_feature_count:
    raise ValueError("Mismatch in number of features after concatenation")

# Check 4: Check for NaNs or infinite values
if np.isnan(combined_features).any() or np.isinf(combined_features).any():
    raise ValueError("NaNs or infinite values found in combined features")


Given the nature of MIMIC III dataset, which is derived from large-scale electronic health record (EHR) databases and involves high-dimensional feature representations, selecting the right machine learning model is key. The choice of model should balance the ability to handle complex, high-dimensional data with interpretability and computational efficiency.

Theoretically Favourable Models
Random Forest: This model is excellent for handling high-dimensional data and does not require scaling of features. It's also good for interpretation as it can provide feature importance.

Gradient Boosting Machines (GBM): Models like XGBoost or LightGBM are powerful for high-dimensional data and often provide better performance than Random Forest, though they might be less interpretable and require careful tuning.

*Neural Networks: Deep learning models can be very effective for complex datasets but require more computational resources. They might be overkill for this task unless I specifically need to model non-linear, complex relationships and interactions in the data. In addition, for neural networks they work with data where the time-dependent and time invariant features are independent instead of combined as shown here. Hence, we will explore them in another notebook.*

Logistic Regression with Regularization: If interpretability is a key concern, logistic regression with L1 or L2 regularization can be a good choice. It's simpler than tree-based models but can still handle high-dimensional data with the right regularization.

Support Vector Machines (SVM): SVM with a linear or non-linear kernel can be effective, especially if the dataset has clear margins of separation. However, they can be computationally intensive for very large datasets.

Model Evaluation
Cross-Validation: Use cross-validation to assess the generalizability of your model.
Metrics: Given this is a healthcare dataset, metrics like accuracy, precision, recall, F1 score, and especially the Area Under the ROC Curve (AUC-ROC) are important.

**Random Forest**

Code Description
n_estimators: This parameter specifies the number of trees in the forest. You can experiment with different values, but 100 is a good starting point.
random_state: Setting a random state ensures reproducibility.
Model Evaluation: The code includes accuracy and ROC AUC metrics, which are standard for classification tasks. The classification report provides a detailed breakdown of precision, recall, and F1-score for each class. ROC AUC, is especially important in binary classification tasks, especially in medical contexts where the balance between sensitivity (true positive rate) and specificity (true negative rate) is important. A value closer to 1 indicates a better model
Feature Importance: Random Forest can provide insights into which features are most important for the prediction. You can access this using random_forest_model.feature_importances_.

In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

# Assuming train_data, train_labels, test_data, test_labels are already defined

# Initialize the Random Forest model
random_forest_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model on the training data
random_forest_model.fit(train_data, train_labels)

# Predict on the test data
test_predictions = random_forest_model.predict(test_data)

# Evaluate the model
accuracy = accuracy_score(test_labels, test_predictions)
roc_auc = roc_auc_score(test_labels, test_predictions)

print(f"Accuracy: {accuracy}")
print(f"ROC AUC: {roc_auc}")
print("Classification Report:")
print(classification_report(test_labels, test_predictions))

# Feature Importance Analysis
importances = random_forest_model.feature_importances_
indices = np.argsort(importances)[::-1]

print("Top Feature rankings:")
for i in range(10):  # Top 10 features; adjust as needed
    print(f"{i + 1}. Feature: {total_feature_names[indices[i]]}, Importance: {importances[indices[i]]}")


Accuracy: 0.8955696202531646
ROC AUC: 0.5133847306992183
Classification Report:
              precision    recall  f1-score   support

           0       0.90      1.00      0.94      1132
           1       0.50      0.03      0.06       132

    accuracy                           0.90      1264
   macro avg       0.70      0.51      0.50      1264
weighted avg       0.86      0.90      0.85      1264

Top Feature rankings:
1. Feature: 226104_value_Unresponsive_time46, Importance: 0.0017349550073114859
2. Feature: 225154_InputRoute: Continuous Med_value_1.0_time46, Importance: 0.0013277409871644932
3. Feature: 225792_value_1.0_time46, Importance: 0.000993736674619391
4. Feature: 225154_InputRoute: Continuous Med_value_1.0_time47, Importance: 0.0009186840100308196
5. Feature: 225792_value_1.0_time34, Importance: 0.0008215305539938766
6. Feature: 221906_value_1.0_time37, Importance: 0.0008183066326337454
7. Feature: 223904_value_Absent_time30, Importance: 0.000745496294177183
8. Feature

**Understanding the Feature Names**
Feature Identifier (e.g., 226104, 225154): These numbers likely correspond to specific measurements, observations, or interventions recorded in the electronic health records. For example, they could be codes for certain lab tests, vital signs, medication administrations, or other clinical data points.

Feature Description (e.g., value, InputRoute): This part describes the nature of the feature. For example, value might indicate the recorded value of a measurement, while InputRoute could specify the route of medication administration (like oral, intravenous, etc.).

Feature Value (e.g., Unresponsive, 1.0, Absent): This represents the actual value or category of the feature. For instance, Unresponsive might indicate the patient's response status, 1.0 could be a binary indicator (such as presence/absence of a condition or response), and Absent might refer to the absence of a certain clinical sign.

Time Point (e.g., time46): This indicates the specific time step in the patient's data timeline. For time-dependent features, each time step (e.g., hour, day) is treated as a separate feature. time46 means the feature's value at the 46th time step (the exact unit depends on how the data was recorded and processed).

Understanding in Context
To fully understand these features, you would need to refer to the documentation of the MIMIC-III and eICU databases or consult with healthcare professionals familiar with these datasets. They can provide the context behind these identifiers and what each feature represents in a clinical setting.

For example, a feature like 226104_value_Unresponsive_time46 might be interpreted as the patient's response status (Unresponsive) recorded at the 46th time step in the dataset.

The feature 225154_InputRoute: Continuous Med_value_1.0_time46 could indicate that at the 46th time step, there was a continuous medication administered (value 1.0 meaning 'Yes' or 'Present').

**Gradient Boosting**

Hyperparameter Selection:
n_estimators (Number of Trees):

Initial Value: 100
Rationale: A larger number of trees can improve the model's ability to capture complex patterns, but also increases the risk of overfitting, especially in a high-dimensional space. Starting with 100 is a balanced choice, allowing us to observe model performance without excessive computational demand.
Adjustment Consideration: If the model underfits, you may increase this number. If it overfits, decreasing it or adding regularization might help.
learning_rate (Shrinkage):

Initial Value: 0.1
Rationale: This controls the contribution of each tree to the final model. A lower rate requires more trees but can lead to a more robust model. 0.1 is a common starting point that balances model complexity and training efficiency.
Adjustment Consideration: If the model trains too slowly or overfits, you might increase this rate. If it underfits, decreasing the rate could help.
max_depth (Depth of Each Tree):

Initial Value: 5
Rationale: Given the high dimensionality, a certain level of depth is required to capture interactions between features. However, too much depth can lead to overfitting. A moderate depth like 5 is a starting point that allows for some complexity without being too prone to overfitting.
Adjustment Consideration: If the model underfits, you can increase the depth. If it overfits, reducing the depth or increasing regularization may be beneficial.


Feature Importance: XGBoost also provides feature importance, which can be useful for understanding the predictive drivers in your dataset.

In [22]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

# Initialize the XGBoost classifier
xgb_model = xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)

# Train the model on the training data
xgb_model.fit(train_data, train_labels)

# Predict on the test data
xgb_predictions = xgb_model.predict(test_data)

# Evaluate the model
accuracy = accuracy_score(test_labels, xgb_predictions)
roc_auc = roc_auc_score(test_labels, xgb_predictions)

print(f"Accuracy: {accuracy}")
print(f"ROC AUC: {roc_auc}")
print("Classification Report:")
print(classification_report(test_labels, xgb_predictions))

# Feature Importance Analysis (optional)
xgb_importances = xgb_model.feature_importances_
# You can use similar logic as before to display feature importances

Accuracy: 0.9098101265822784
ROC AUC: 0.6317592890031052
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.98      0.95      1132
           1       0.66      0.28      0.39       132

    accuracy                           0.91      1264
   macro avg       0.79      0.63      0.67      1264
weighted avg       0.89      0.91      0.89      1264


In [24]:
# Get feature importances from the trained XGBoost model
xgb_importances = xgb_model.feature_importances_

# Sort the feature importances in descending order
indices = np.argsort(xgb_importances)[::-1]

# Display the top 10 feature importances
print("Top Feature rankings:")
for i in range(10):  # Adjust the range as needed to display more or fewer features
    feature_name = total_feature_names[indices[i]] if indices[i] < len(total_feature_names) else "Feature Index Out of Bound"
    print(f"{i + 1}. Feature: {feature_name}, Importance: {xgb_importances[indices[i]]}")


Top Feature rankings:
1. Feature: 225792_value_1.0_time47, Importance: 0.020830070599913597
2. Feature: 225792_value_1.0_time46, Importance: 0.012420390732586384
3. Feature: 225154_InputRoute: Continuous Med_value_1.0_time47, Importance: 0.008466214872896671
4. Feature: 225158_value_1.0_time47, Importance: 0.005562117788940668
5. Feature: 226559_value_(-0.001, 30.0]_time40, Importance: 0.005055662710219622
6. Feature: 225158_InputRoute: Continuous Med_value_1.0_time42, Importance: 0.005000233184546232
7. Feature: 226104_value_Unresponsive_time46, Importance: 0.004466429818421602
8. Feature: 225158_value_1.0_time42, Importance: 0.004417689982801676
9. Feature: 221906_value_1.0_time47, Importance: 0.0038532603066414595
10. Feature: 221906_value_1.0_time34, Importance: 0.0036179297603666782


Interpretation of the Results
Time-Dependent Features: Similar to the Random Forest model, the XGBoost model also emphasizes the importance of time-dependent features. This suggests that certain conditions or observations at specific time points are critical in determining the outcome.

Feature Specifics: The features identified (e.g., 225792_value_1.0_time47, 226104_value_Unresponsive_time46) correspond to specific measurements, statuses, or interventions recorded in the EHR data. Their significance might reflect clinical conditions or treatments that are particularly relevant to patient outcomes at those times.

Clinical Relevance: Each feature likely has a clinical interpretation. For instance, 225792_value_1.0_time47 might indicate a particular measurement or status that is crucial at the 47th time step. The importance of Unresponsive status at a certain time point could be indicative of critical patient conditions.

Value Ranges: Some features include value ranges (e.g., (-0.001, 30.0]), which might represent discretized or binned values of continuous variables.

**Logistic Regression with L1 Regularization (Lasso)**
L1 regularization tends to push coefficients of less important features to exactly zero, thus performing feature selection.



Considerations:
Regularization Strength: The C parameter in LogisticRegression controls the strength of regularization. Lower values of C specify stronger regularization. You might need to experiment with this parameter to find the best value.

Solver: The choice of solver is important especially for large datasets. For L1 regularization, liblinear is a good choice. For L2, you might also consider lbfgs or sag for larger datasets.

Max Iterations (max_iter): Depending on your dataset's size and complexity, you might need to increase the number of iterations.

Scaling Features: Logistic Regression can benefit from feature scaling, especially if regularization is used.

Interpretability: After training, you can examine the coefficients of the model to understand the influence of each feature. Features with larger absolute coefficients are more influential in predicting the outcome.

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

# Initialize the Logistic Regression model with L1 regularization
log_reg_l1 = LogisticRegression(penalty='l1', solver='liblinear', random_state=42, max_iter=100)

# Train the model
log_reg_l1.fit(train_data, train_labels)

# Predict on the test data
log_reg_l1_predictions = log_reg_l1.predict(test_data)

# Evaluate the model
accuracy_l1 = accuracy_score(test_labels, log_reg_l1_predictions)
roc_auc_l1 = roc_auc_score(test_labels, log_reg_l1_predictions)

print(f"Accuracy (L1): {accuracy_l1}")
print(f"ROC AUC (L1): {roc_auc_l1}")
print("Classification Report (L1):")
print(classification_report(test_labels, log_reg_l1_predictions))


Accuracy (L1): 0.8916139240506329
ROC AUC (L1): 0.6584082878252491
Classification Report (L1):
              precision    recall  f1-score   support

           0       0.93      0.95      0.94      1132
           1       0.48      0.36      0.41       132

    accuracy                           0.89      1264
   macro avg       0.70      0.66      0.68      1264
weighted avg       0.88      0.89      0.89      1264


**Logistic Regression with L2 Regularization (Ridge)**
L2 regularization tends to shrink the coefficients of less important features, but does not set them to zero.

In [26]:
# Initialize the Logistic Regression model with L2 regularization
log_reg_l2 = LogisticRegression(penalty='l2', solver='liblinear', random_state=42, max_iter=100)

# Train the model
log_reg_l2.fit(train_data, train_labels)

# Predict on the test data
log_reg_l2_predictions = log_reg_l2.predict(test_data)

# Evaluate the model
accuracy_l2 = accuracy_score(test_labels, log_reg_l2_predictions)
roc_auc_l2 = roc_auc_score(test_labels, log_reg_l2_predictions)

print(f"Accuracy (L2): {accuracy_l2}")
print(f"ROC AUC (L2): {roc_auc_l2}")
print("Classification Report (L2):")
print(classification_report(test_labels, log_reg_l2_predictions))


Accuracy (L2): 0.8884493670886076
ROC AUC (L2): 0.6198334939501017
Classification Report (L2):
              precision    recall  f1-score   support

           0       0.92      0.96      0.94      1132
           1       0.45      0.28      0.34       132

    accuracy                           0.89      1264
   macro avg       0.68      0.62      0.64      1264
weighted avg       0.87      0.89      0.88      1264


**SVM**
Considerations
Calibration of Probabilities: LinearSVC doesn't output probabilities by default, which are needed for ROC AUC. CalibratedClassifierCV is used to calibrate probabilities in a post-processing step.

Maximum Iterations (max_iter): This parameter may need adjustment. Increase it if the algorithm doesn't converge, but be aware that higher values will increase computation time.

Handling Large Datasets: Linear SVMs are generally more scalable for large datasets, but if you still face computational issues, consider using a subset of the data or feature selection techniques to reduce dimensionality.

Non-linear SVM: If the linear SVM does not perform adequately, exploring non-linear kernels like RBF might be beneficial, though at a higher computational cost.



In [27]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from sklearn.calibration import CalibratedClassifierCV

# Initialize the Linear SVM model
linear_svc = LinearSVC(random_state=42, max_iter=1000)

# Calibrate probabilities
svm_calibrated = CalibratedClassifierCV(linear_svc) 

# Train the model
svm_calibrated.fit(train_data, train_labels)

# Predict on the test data
svm_predictions = svm_calibrated.predict(test_data)
svm_probabilities = svm_calibrated.predict_proba(test_data)[:, 1]  # Probabilities for ROC AUC

# Evaluate the model
accuracy = accuracy_score(test_labels, svm_predictions)
roc_auc = roc_auc_score(test_labels, svm_probabilities)

print(f"Accuracy: {accuracy}")
print(f"ROC AUC: {roc_auc}")
print("Classification Report:")
print(classification_report(test_labels, svm_predictions))


/Users/ashishsaragadam/anaconda3/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/ashishsaragadam/anaconda3/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/ashishsaragadam/anaconda3/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/ashishsaragadam/anaconda3/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
 

Accuracy: 0.9003164556962026
ROC AUC: 0.8151434843130956
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.98      0.95      1132
           1       0.57      0.18      0.28       132

    accuracy                           0.90      1264
   macro avg       0.74      0.58      0.61      1264
weighted avg       0.88      0.90      0.88      1264


**RNN**

So remember how we reshaped and combined the time-dependent (X) and time-invariant (s) features into a single 2D matrix (combined_features)? This format was suitable for models like Random Forest, XGBoost, and Logistic Regression.

'# Reshape X from 3D tensor to 2D matrix
# The new shape should be (number of patients, time * features)
# Assuming the first dimension of X is the number of patients
num_patients = X.shape[0]
time_steps = X.shape[1]
num_features_X = X.shape[2]

X_reshaped = X.reshape(num_patients, time_steps * num_features_X)

# Concatenate s (time-invariant) and X_reshaped (time-dependent) features
# Ensure that both s and X_reshaped have the same number of rows (patients)
combined_features = np.concatenate([s, X_reshaped], axis=1)'

However, for an RNN, we need to maintain the time-dependent features in their original 3D tensor format and handle the time-invariant features differently, as RNNs are designed to process sequential data. Let's adjust the preparation of your data for the RNN model.

Adjusting Data for RNN
Revert to Original 3D Tensor for Time-Dependent Features: Need to use X in its original (num_patients, time_steps, num_features_X) format.

Repeat Time-Invariant Features for Each Time Step: Repeat s for each time step and concatenate it with X along the feature dimension.

In [40]:
# Assuming X is your time-dependent data (3D tensor) and s is your time-invariant data (2D matrix)

# Repeat time-invariant features (s) for each time step
s_repeated = np.repeat(s[:, np.newaxis, :], time_steps, axis=1)

# Concatenate with time-dependent features (X) along the feature dimension
rnn_input = np.concatenate([X, s_repeated], axis=-1)

# Now, 'rnn_input' is in the correct format for RNN input
print(f"Shape of time-dependent features (X): {X.shape}")
print(f"Shape of repeated time-invariant features (s): {s_repeated.shape}")
print(f"Shape of RNN input: {rnn_input.shape}")


Shape of time-dependent features (X): (8577, 48, 7307)
Shape of repeated time-invariant features (s): (8577, 48, 96)
Shape of RNN input: (8577, 48, 7403)


What does the output above mean?

Time-Dependent Features (X): (8577, 48, 7307)

You have 8,577 patients.
Each patient has data recorded over 48 time steps.
There are 7,307 features per time step.

Repeated Time-Invariant Features (s): (8577, 48, 96)
The 96 time-invariant features are repeated for each of the 48 time steps.

Combined RNN Input: (8577, 48, 7403)
Each patient's data is now a sequence of 48 time steps.
At each time step, there are 7,403 features (7,307 time-dependent + 96 time-invariant).



Since the format of the data for the RNN is different from what we used for the other models, we need to adjust the way you partition the data into training, testing, and validation sets.

For the RNN, we have the rnn_input tensor that combines both the time-dependent and time-invariant features appropriately. we need to split this tensor along with the labels according to the partitions indicated in df_pop.

In [32]:
# Create boolean masks for train, test, and validation partitions
train_mask = df_pop['partition'] == 'train'
test_mask = df_pop['partition'] == 'test'
val_mask = df_pop['partition'] == 'val' 

# Split the rnn_input tensor and labels according to the partitions
rnn_input_train = rnn_input[train_mask]
rnn_input_test = rnn_input[test_mask]
rnn_input_val = rnn_input[val_mask]  

train_labels = df_pop[train_mask]['mortality_LABEL']
test_labels = df_pop[test_mask]['mortality_LABEL']
val_labels = df_pop[val_mask]['mortality_LABEL']  


In [34]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from sklearn.metrics import roc_auc_score
from tensorflow.keras.callbacks import Callback

# Custom callback to compute ROC AUC after each epoch
class RocAucCallback(Callback):
    def __init__(self, training_data, validation_data):
        self.x_train, self.y_train = training_data
        self.x_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs=None):
        y_train_pred = self.model.predict(self.x_train)
        y_val_pred = self.model.predict(self.x_val)
        roc_auc_train = roc_auc_score(self.y_train, y_train_pred)
        roc_auc_val = roc_auc_score(self.y_val, y_val_pred)
        print(f'Epoch {epoch+1}: roc_auc_train: {roc_auc_train}, roc_auc_val: {roc_auc_val}')

# Define the RNN model
model = Sequential()
model.add(SimpleRNN(50, return_sequences=True, input_shape=(48, 7403)))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Instantiate the RocAucCallback
rocauc_callback = RocAucCallback(training_data=(rnn_input_train, train_labels),
                                 validation_data=(rnn_input_val, val_labels))

# Train the model with validation data and the custom callback
model.fit(rnn_input_train, train_labels, epochs=10, batch_size=64, 
          validation_data=(rnn_input_val, val_labels), callbacks=[rocauc_callback])

# Evaluate the model on the test set
loss, accuracy = model.evaluate(rnn_input_test, test_labels)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

# Calculate and print ROC AUC for the test set
test_predictions = model.predict(rnn_input_test)
roc_auc_test = roc_auc_score(test_labels, test_predictions)
print(f"Test ROC AUC: {roc_auc_test}")


Epoch 1/10
40/40 [==============================] - 1s 16ms/step


ValueError: Found array with dim 3. None expected <= 2.

Here we are left with a dilemma

The error you encountered, ValueError: Found array with dim 3. None expected <= 2, is due to the roc_auc_score function expecting 1D array predictions and true labels, but the predictions made by your RNN model are 3D due to the return_sequences=True parameter in the SimpleRNN layer.


To resolve this issue, we have two main options:

Change return_sequences to False: This will make the RNN output predictions only for the last time step. It's the standard approach for many sequence classification tasks where the final prediction is based on the entire sequence.

Aggregate the Predictions Across Time Steps: If our task requires predictions at each time step, that is return_sequences has to be True, then we will need to aggregate these predictions into a single prediction per sample before calculating ROC AUC.



return_sequences=True
Objective: The model predicts an output at each time step in the sequence. This is useful in scenarios where you need a prediction for each moment in time, not just a single prediction based on the entire sequence.

Output: The output is a 3D tensor with dimensions (samples, time_steps, features). In a binary classification scenario, the third dimension usually represents the probability of belonging to one of the classes for each time step.

Use Case Example: A common example is in natural language processing tasks like part-of-speech tagging, where you want to tag each word (time step) in a sentence (sequence). In a healthcare context, this might translate to making a diagnosis or prediction at each time interval based on the data up to that point.

return_sequences=False
Objective: The model makes a single prediction for the entire sequence. This is suitable for tasks where the outcome is dependent on the entire sequence, and you're interested in the final output at the last time step.

Output: The output is a 2D tensor with dimensions (samples, features). For binary classification, this typically translates to a single probability score per sample.

Use Case Example: An example could be sentiment analysis where the sentiment of the entire sentence (sequence) is determined. In healthcare, this might be used for predicting patient outcomes (like mortality or readmission) based on the entire sequence of their EHR data.

Choosing the Right Approach for Your Task
If your goal with the EHR dataset is to predict in-hospital mortality based on the entire sequence of recorded data (i.e., the prediction is about the final outcome at the end of the observed period), return_sequences=False is appropriate.

If, however, you are interested in how the risk of mortality evolves at each time step (e.g., hour by hour prediction), then return_sequences=True would be the correct choice. This would provide a more granular view, potentially offering insights into how a patient's risk profile changes over time.

Although keeping return_sequences = True might be the more interesting choice for me personally, our initial objective and the objective we used for all the other models aligns with changing to return_sequences=False

In [35]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from sklearn.metrics import roc_auc_score
from tensorflow.keras.callbacks import Callback

# Custom callback to compute ROC AUC after each epoch
class RocAucCallback(Callback):
    def __init__(self, training_data, validation_data):
        self.x_train, self.y_train = training_data
        self.x_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs=None):
        y_train_pred = self.model.predict(self.x_train)
        y_val_pred = self.model.predict(self.x_val)
        roc_auc_train = roc_auc_score(self.y_train, y_train_pred)
        roc_auc_val = roc_auc_score(self.y_val, y_val_pred)
        print(f'Epoch {epoch+1}: roc_auc_train: {roc_auc_train}, roc_auc_val: {roc_auc_val}')

# Define the RNN model
model = Sequential()
model.add(SimpleRNN(50, return_sequences=False, input_shape=(48, 7403)))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Instantiate the RocAucCallback
rocauc_callback = RocAucCallback(training_data=(rnn_input_train, train_labels),
                                 validation_data=(rnn_input_val, val_labels))

# Train the model with validation data and the custom callback
model.fit(rnn_input_train, train_labels, epochs=10, batch_size=64, 
          validation_data=(rnn_input_val, val_labels), callbacks=[rocauc_callback])

# Evaluate the model on the test set
loss, accuracy = model.evaluate(rnn_input_test, test_labels)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

# Calculate and print ROC AUC for the test set
test_predictions = model.predict(rnn_input_test)
roc_auc_test = roc_auc_score(test_labels, test_predictions)
print(f"Test ROC AUC: {roc_auc_test}")

Epoch 1/10
40/40 [==============================] - 1s 16ms/step
Epoch 1: roc_auc_train: 0.9331223660445515, roc_auc_val: 0.8586860670194003
95/95 [==============================] - 18s 191ms/step - loss: 0.3125 - accuracy: 0.8767 - val_loss: 0.2577 - val_accuracy: 0.8936
Epoch 2/10
40/40 [==============================] - 1s 16ms/step
Epoch 2: roc_auc_train: 0.9853243527995185, roc_auc_val: 0.853407309425828
95/95 [==============================] - 21s 227ms/step - loss: 0.1990 - accuracy: 0.9210 - val_loss: 0.2602 - val_accuracy: 0.8975
Epoch 3/10
40/40 [==============================] - 1s 22ms/step
Epoch 3: roc_auc_train: 0.9955614087898855, roc_auc_val: 0.8407615618263766
95/95 [==============================] - 22s 230ms/step - loss: 0.1142 - accuracy: 0.9611 - val_loss: 0.3089 - val_accuracy: 0.8795
Epoch 4/10
40/40 [==============================] - 1s 15ms/step
Epoch 4: roc_auc_train: 0.9997145293999599, roc_auc_val: 0.8148331863609641
95/95 [==============================] - 

Training and Validation Performance
Accuracy and Loss over Epochs:

Training Accuracy increases significantly over epochs, reaching near-perfect levels. This suggests that the model is learning to classify the training data very effectively.
Validation Accuracy also increases but plateaus and remains lower than the training accuracy. This indicates that while the model is learning generalizable patterns, there might be some overfitting to the training data.
Training Loss decreases rapidly, which is expected as the model learns.
Validation Loss initially decreases but then starts to increase, which is a classic sign of overfitting.
ROC AUC Scores:

Training ROC AUC is very high, close to 1.0, which suggests excellent performance on the training set.
Validation ROC AUC is lower than the training ROC AUC and shows some fluctuation. While it's still a good score, the gap between training and validation indicates the model may be overfitting.
Test Performance
Test Accuracy: The accuracy on the test set is 0.8884, which is good but lower than the training accuracy, supporting the indication of overfitting.
Test ROC AUC: A score of 0.8207 is quite strong, indicating that the model has good discriminative ability on the test data.
Interpretation and Next Steps
Overfitting: The model seems to be overfitting the training data. This is evident from the high training accuracy and ROC AUC, and the growing gap between training and validation loss. Regularization techniques, dropout, or more sophisticated RNN layers like LSTM or GRU could help mitigate this.

Model Complexity: You might need to adjust the model's complexity. If it's too complex for the data, simplifying the model could help improve generalizability.

Hyperparameter Tuning: Experimenting with different learning rates, number of epochs, batch sizes, and other hyperparameters could yield better performance.

Data Representation: Consider if the way you're feeding data into the model is optimal. Since RNNs are sensitive to the sequence length and feature representation, adjustments here might be beneficial.

Further Evaluation: Dive deeper into the types of errors the model is making. Confusion matrices, precision-recall curves, and analyzing misclassified examples can provide more insights.

Remember, while accuracy and ROC AUC are important metrics, they don't always tell the whole story, especially in imbalanced datasets common in healthcare contexts. Balancing sensitivity and specificity is crucial, and focusing on clinical relevance is key.

Addressing overfitting in an RNN model, especially when dealing with a complex and high-dimensional dataset involves a combination of strategies. Here are some approaches to consider:

3. Regularization
Add Dropout: Dropout layers randomly set a fraction of input units to 0 at each update during training, which helps prevent overfitting. You can add them between RNN layers or Dense layers.
L1/L2 Regularization: Apply L1 or L2 regularization to the weights of the network layers. This can be done through the kernel_regularizer parameter in the layers. This did not yield great metrics when applied with linear regression so did not apply. 
2. Model Complexity
Reduce Model Size: Simplify your model by reducing the number of neurons in each layer or reducing the number of layers. A smaller model is less prone to overfitting. I wanted to try this strategy but first wanted to use LSTM or GRU layers to see how the metrics change if we actually increase the complexity given that they are long-term dependencies. 
3. Early Stopping
Implement early stopping during training. This stops training when the model performance stops improving on a validation dataset.
4. Advanced RNN Layers
Use LSTM or GRU Layers: These layers are more complex than simple RNN layers and are better at capturing long-term dependencies without overfitting.
7. Hyperparameter Tuning
Experiment with different learning rates, batch sizes, or other hyperparameters.


**Changes Implemented to the RNN Model to Address Overfitting**

GRU Layer: You've replaced the SimpleRNN layer with a GRU layer, which is generally better at capturing long-term dependencies in the data.

Dropout: The addition of a Dropout layer (with a dropout rate of 0.2) will help in preventing overfitting by randomly setting a fraction of the input units to 0 at each update during training.

Early Stopping: You've included an EarlyStopping callback, which will stop the training process if the model's performance on the validation set doesn't improve for 3 consecutive epochs (patience=3). This prevents overfitting by not allowing the model to train excessively.


In [37]:



import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from sklearn.metrics import roc_auc_score
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.layers import Dropout, GRU
from tensorflow.keras.callbacks import EarlyStopping

# Custom callback to compute ROC AUC after each epoch
class RocAucCallback(Callback):
    def __init__(self, training_data, validation_data):
        self.x_train, self.y_train = training_data
        self.x_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs=None):
        y_train_pred = self.model.predict(self.x_train)
        y_val_pred = self.model.predict(self.x_val)
        roc_auc_train = roc_auc_score(self.y_train, y_train_pred)
        roc_auc_val = roc_auc_score(self.y_val, y_val_pred)
        print(f'Epoch {epoch+1}: roc_auc_train: {roc_auc_train}, roc_auc_val: {roc_auc_val}')



# Define the RNN model with Dropout and GRU
model = Sequential()
model.add(GRU(50, return_sequences=False, input_shape=(48, 7403)))
model.add(Dropout(0.2))  # Dropout layer
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# Train the model with validation data and callbacks for early stopping
model.fit(rnn_input_train, train_labels, epochs=10, batch_size=64, 
          validation_data=(rnn_input_val, val_labels), callbacks=[rocauc_callback, early_stopping])


# Evaluate the model on the test set
loss, accuracy = model.evaluate(rnn_input_test, test_labels)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

# Calculate and print ROC AUC for the test set
test_predictions = model.predict(rnn_input_test)
roc_auc_test = roc_auc_score(test_labels, test_predictions)
print(f"Test ROC AUC: {roc_auc_test}")


Epoch 1/10
40/40 [==============================] - 1s 21ms/step
Epoch 1: roc_auc_train: 0.9233619305639174, roc_auc_val: 0.8726055751518715
95/95 [==============================] - 28s 284ms/step - loss: 0.3141 - accuracy: 0.8756 - val_loss: 0.2442 - val_accuracy: 0.9085
Epoch 2/10
40/40 [==============================] - 1s 20ms/step
Epoch 2: roc_auc_train: 0.9708604254465181, roc_auc_val: 0.8746448167744464
95/95 [==============================] - 26s 272ms/step - loss: 0.2088 - accuracy: 0.9160 - val_loss: 0.2474 - val_accuracy: 0.9006
Epoch 3/10
40/40 [==============================] - 1s 21ms/step
Epoch 3: roc_auc_train: 0.9865467589805338, roc_auc_val: 0.8688577797374093
95/95 [==============================] - 26s 276ms/step - loss: 0.1539 - accuracy: 0.9422 - val_loss: 0.2668 - val_accuracy: 0.9030
Epoch 4/10
40/40 [==============================] - 1s 22ms/step
Epoch 4: roc_auc_train: 0.994092915914108, roc_auc_val: 0.8531501077797375
40/40 [==============================] - 

Training and Validation Performance Over Epochs
Accuracy and Loss:

Training Accuracy increases over epochs, indicating the model is learning to classify the training data effectively.
Validation Accuracy also increases initially but doesn't reach the same level as training accuracy and slightly decreases in the later epochs. This suggests some overfitting to the training data.
Training Loss decreases consistently, which is expected as the model learns.
Validation Loss initially decreases but then starts increasing, a sign of overfitting. The model performs well initially but begins to lose generalizability in later epochs.
ROC AUC Scores:

Training ROC AUC improves steadily, reaching a very high score, which suggests excellent performance on the training set.
Validation ROC AUC is relatively high initially but begins to decline. This discrepancy between training and validation ROC AUC is another indicator of overfitting.
Test Performance
Test Accuracy: The accuracy on the test set is 0.875, which is fairly good but noticeably lower than the peak training accuracy. This gap further supports the overfitting indication.
Test ROC AUC: A score of 0.8594 is quite good and indicates that the model has a strong ability to distinguish between the classes. However, this score is still lower than what the model achieved on the training set.
Overall Interpretation
The model learns to fit the training data well, but it starts to overfit as indicated by the increasing validation loss and the decreasing validation ROC AUC in later epochs.
The test performance, while good, doesn't match the model's performance on the training set, which is consistent with the overfitting pattern observed during training.
The ROC AUC scores are a highlight, indicating that the model has a good discriminative ability, but there's still room for improvement, especially in terms of generalizability.

**Changes to RNN to address the persistent overfitting**
Hyperparameter Tuning: Adjusting the architecture by using LSTM layers instead could yield better results.




In [39]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import roc_auc_score
from tensorflow.keras.callbacks import Callback, EarlyStopping

# Custom callback to compute ROC AUC after each epoch
class RocAucCallback(Callback):
    def __init__(self, training_data, validation_data):
        self.x_train, self.y_train = training_data
        self.x_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs=None):
        y_train_pred = self.model.predict(self.x_train)
        y_val_pred = self.model.predict(self.x_val)
        roc_auc_train = roc_auc_score(self.y_train, y_train_pred)
        roc_auc_val = roc_auc_score(self.y_val, y_val_pred)
        print(f'Epoch {epoch+1}: roc_auc_train: {roc_auc_train}, roc_auc_val: {roc_auc_val}')

# Define the RNN model with an LSTM layer
model = Sequential()
model.add(LSTM(50, return_sequences=False, input_shape=(48, 7403)))
model.add(Dropout(0.2))  # Dropout layer
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# Train the model with validation data and callbacks for early stopping
model.fit(rnn_input_train, train_labels, epochs=10, batch_size=64, 
          validation_data=(rnn_input_val, val_labels), callbacks=[rocauc_callback, early_stopping])

# Evaluate the model on the test set
loss, accuracy = model.evaluate(rnn_input_test, test_labels)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

# Calculate and print ROC AUC for the test set
test_predictions = model.predict(rnn_input_test)
roc_auc_test = roc_auc_score(test_labels, test_predictions)
print(f"Test ROC AUC: {roc_auc_test}")


Epoch 1/10
40/40 [==============================] - 1s 22ms/step
Epoch 1: roc_auc_train: 0.926501856311459, roc_auc_val: 0.8825262100725063
95/95 [==============================] - 29s 294ms/step - loss: 0.3058 - accuracy: 0.8795 - val_loss: 0.2438 - val_accuracy: 0.9077
Epoch 2/10
40/40 [==============================] - 1s 23ms/step
Epoch 2: roc_auc_train: 0.9663689042745334, roc_auc_val: 0.8747427983539094
95/95 [==============================] - 28s 292ms/step - loss: 0.2157 - accuracy: 0.9155 - val_loss: 0.2472 - val_accuracy: 0.9069
Epoch 3/10
40/40 [==============================] - 1s 21ms/step
Epoch 3: roc_auc_train: 0.9822353501906481, roc_auc_val: 0.8627155594748187
95/95 [==============================] - 28s 293ms/step - loss: 0.1624 - accuracy: 0.9345 - val_loss: 0.2688 - val_accuracy: 0.8983
Epoch 4/10
40/40 [==============================] - 1s 21ms/step
Epoch 4: roc_auc_train: 0.9945459562512543, roc_auc_val: 0.8474794238683128
40/40 [==============================] - 

**Adapting Existing Top Performing Models to my Preprocessed Data**

There are top performing models out there with the hopes of being used as benchmarks. However, because the FIDDLE preprocessing is a more robust preprocessing technique than employed for the previous benchmarks, there are no models built on this specific preprocessed dataset. Hence, the next two models use the architecture from existing benchmark models and adapted to the current dataset and preprocessing functions to ensure good scientific practice.


First Benchmark Model (LSTM):
Source: https://github.com/YerevaNN/mimic3-benchmarks

The model below has the same architecture to the model we built from scratch, indicating that we did quite well in reaching the standard benchmark for in-hospital mortality from scratch using a more robust preprocessing technique. 

In [41]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import roc_auc_score
from tensorflow.keras.callbacks import Callback, EarlyStopping

# Custom callback to compute ROC AUC after each epoch
class RocAucCallback(Callback):
    def __init__(self, training_data, validation_data):
        self.x_train, self.y_train = training_data
        self.x_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs=None):
        y_train_pred = self.model.predict(self.x_train)
        y_val_pred = self.model.predict(self.x_val)
        roc_auc_train = roc_auc_score(self.y_train, y_train_pred)
        roc_auc_val = roc_auc_score(self.y_val, y_val_pred)
        print(f'Epoch {epoch+1}: roc_auc_train: {roc_auc_train}, roc_auc_val: {roc_auc_val}')

# Define the RNN model with an LSTM layer
model = Sequential()
model.add(LSTM(50, return_sequences=False, input_shape=(48, 7403)))  # Adjust the input shape as needed
model.add(Dropout(0.2))  # Dropout layer to prevent overfitting
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping callback to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# Train the model with validation data and callbacks for early stopping
model.fit(rnn_input_train, train_labels, epochs=10, batch_size=64, 
          validation_data=(rnn_input_val, val_labels), callbacks=[rocauc_callback, early_stopping])

# Evaluate the model on the test set
loss, accuracy = model.evaluate(rnn_input_test, test_labels)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

# Calculate and print ROC AUC for the test set
test_predictions = model.predict(rnn_input_test)
roc_auc_test = roc_auc_score(test_labels, test_predictions)
print(f"Test ROC AUC: {roc_auc_test}")


Epoch 1/10
40/40 [==============================] - 1s 21ms/step
Epoch 1: roc_auc_train: 0.9234958860124423, roc_auc_val: 0.8815525181265923
95/95 [==============================] - 29s 295ms/step - loss: 0.3077 - accuracy: 0.8807 - val_loss: 0.2408 - val_accuracy: 0.9069
Epoch 2/10
40/40 [==============================] - 1s 22ms/step
Epoch 2: roc_auc_train: 0.9710412903873169, roc_auc_val: 0.8793969233784048
95/95 [==============================] - 27s 283ms/step - loss: 0.2099 - accuracy: 0.9150 - val_loss: 0.2549 - val_accuracy: 0.9100
Epoch 3/10
40/40 [==============================] - 1s 21ms/step
Epoch 3: roc_auc_train: 0.9886115291992775, roc_auc_val: 0.8761880266509896
95/95 [==============================] - 27s 285ms/step - loss: 0.1537 - accuracy: 0.9413 - val_loss: 0.2624 - val_accuracy: 0.9038
Epoch 4/10
40/40 [==============================] - 1s 22ms/step
Epoch 4: roc_auc_train: 0.9970226269315673, roc_auc_val: 0.8670389966686263
40/40 [==============================] -

**Second Benchmark Test**
Source: https://paperswithcode.com/paper/early-hospital-mortality-prediction-using

Below is a benchmark that supposedly yields the best results; however, there is no difference in results or architecture in the benchmark-adapted random forest (thats supposed to beat all models for in hospital mortality) and the random forest we created from scratch. Both have poor performance due to a low AUROC score as can be observed below. 

In [43]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
import numpy as np

# Initialize the Random Forest model
random_forest_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model on the training data
# Assuming your training data is a combination of time-dependent and time-independent features
random_forest_model.fit(train_data, train_labels)

# Predict on the test data
test_predictions = random_forest_model.predict(test_data)

# Evaluate the model
accuracy = accuracy_score(test_labels, test_predictions)
roc_auc = roc_auc_score(test_labels, test_predictions)

print(f"Accuracy: {accuracy}")
print(f"ROC AUC: {roc_auc}")
print("Classification Report:")
print(classification_report(test_labels, test_predictions))

# Feature Importance Analysis
importances = random_forest_model.feature_importances_
indices = np.argsort(importances)[::-1]

# Assuming 'total_feature_names' is a list of your feature names
print("Top Feature rankings:")
for i in range(10):  # Adjust the range if you want more or fewer features
    print(f"{i + 1}. Feature: {total_feature_names[indices[i]]}, Importance: {importances[indices[i]]}")



Accuracy: 0.8955696202531646
ROC AUC: 0.5133847306992183
Classification Report:
              precision    recall  f1-score   support

           0       0.90      1.00      0.94      1132
           1       0.50      0.03      0.06       132

    accuracy                           0.90      1264
   macro avg       0.70      0.51      0.50      1264
weighted avg       0.86      0.90      0.85      1264

Top Feature rankings:
1. Feature: 226104_value_Unresponsive_time46, Importance: 0.0017349550073114859
2. Feature: 225154_InputRoute: Continuous Med_value_1.0_time46, Importance: 0.0013277409871644932
3. Feature: 225792_value_1.0_time46, Importance: 0.000993736674619391
4. Feature: 225154_InputRoute: Continuous Med_value_1.0_time47, Importance: 0.0009186840100308196
5. Feature: 225792_value_1.0_time34, Importance: 0.0008215305539938766
6. Feature: 221906_value_1.0_time37, Importance: 0.0008183066326337454
7. Feature: 223904_value_Absent_time30, Importance: 0.000745496294177183
8. Feature